In [5]:
from pathlib import Path

import pandas as pd

# Project paths
PROJECT_ROOT = Path("..").resolve()
DATA_RAW = PROJECT_ROOT / "data" / "raw"
DATA_PROCESSED = PROJECT_ROOT / "data" / "processed"

DATA_PROCESSED.mkdir(parents=True, exist_ok=True)


In [6]:
def clean_basic(df: pd.DataFrame) -> pd.DataFrame:
    """Strip whitespace from key string columns and coerce Age numeric."""
    for col in ["Player", "Nation", "Pos"]:
        if col in df.columns:
            df[col] = df[col].astype(str).str.strip()
    if "Age" in df.columns:
        df["Age"] = pd.to_numeric(df["Age"], errors="coerce")
    return df


def load_standard(path: Path) -> pd.DataFrame:
    """
    Load the Standard Stats table, which has a 2-row header
    like 'Playing Time / Performance / Per 90 Minutes'.
    We flatten it to a simple one-row header and keep the key columns.
    """
    std = pd.read_csv(path, header=[0, 1])

    # Flatten the multiindex columns:
    # use the lower (second) level when it exists; otherwise fall back to the top.
    flat_cols = []
    for top, bottom in std.columns:
        if isinstance(bottom, str) and bottom and not bottom.startswith("Unnamed"):
            flat_cols.append(bottom)
        else:
            flat_cols.append(top)

    std.columns = flat_cols

    # Drop columns that are completely empty
    std = std.dropna(axis=1, how="all")

    # Keep only the columns we care about (filter to what actually exists)
    keep_cols = [
        "Player",
        "Nation",
        "Pos",
        "Age",
        "Gls",
        "Ast",
        "G+A",
        "G-PK",
        "PK",
        "PKatt",
    ]
    keep_cols = [c for c in keep_cols if c in std.columns]

    std = std[keep_cols]
    std = clean_basic(std)

    return std


In [7]:
# Load raw tables
standard = load_standard(DATA_RAW / "manutd_2007_08_standard_stats_sample.csv")

shooting = pd.read_csv(DATA_RAW / "manutd_2007_08_shooting.csv")
goalkeeping = pd.read_csv(DATA_RAW / "manutd_2007_08_goalkeeping.csv")
playing_time = pd.read_csv(DATA_RAW / "manutd_2007_08_playing_time.csv")
misc = pd.read_csv(DATA_RAW / "manutd_2007_08_misc.csv")

# Basic cleaning on the simple tables
shooting = clean_basic(shooting)
goalkeeping = clean_basic(goalkeeping)
playing_time = clean_basic(playing_time)
misc = clean_basic(misc)

# Quick sanity checks
print("STANDARD:", standard.shape)
print("SHOOTING:", shooting.shape)
print("GOALKEEPING:", goalkeeping.shape)
print("PLAYING TIME:", playing_time.shape)
print("MISC:", misc.shape)

standard.head()


STANDARD: (6, 2)
SHOOTING: (25, 17)
GOALKEEPING: (3, 24)
PLAYING TIME: (25, 22)
MISC: (25, 18)


,G+A,G-PK
0,0.09,0.06
1,0.09,0.03
2,0.03,0.00
3,0.07,0.03
4,1.21,0.88


In [11]:
# -----------------------------
# CLEAN + MERGE ALL TABLES
# -----------------------------

# Base table = playing time
master = playing_time.copy()

# -----------------------------
# Clean column names & Player field
# -----------------------------
for df in [master, shooting, goalkeeping, misc]:
    df.columns = df.columns.str.strip()
    if "Player" in df.columns:
        df["Player"] = df["Player"].astype(str).str.strip()

# -----------------------------
# Merge SHOOTING
# -----------------------------
shooting_drop = ["Nation", "Pos", "Age", "90s", "Matches"]
shooting_drop = [c for c in shooting_drop if c in shooting.columns]

master = master.merge(
    shooting.drop(columns=shooting_drop),
    on="Player",
    how="left",
    suffixes=("", "_shoot"),
)

# -----------------------------
# Merge GOALKEEPING
# -----------------------------
gk_drop = ["Nation", "Pos", "Age", "MP", "Starts", "Min", "90s", "Matches"]
gk_drop = [c for c in gk_drop if c in goalkeeping.columns]

master = master.merge(
    goalkeeping.drop(columns=gk_drop),
    on="Player",
    how="left",
    suffixes=("", "_gk"),
)

# -----------------------------
# Merge MISC
# -----------------------------
misc_drop = ["Nation", "Pos", "Age", "90s", "Matches"]
misc_drop = [c for c in misc_drop if c in misc.columns]

master = master.merge(
    misc.drop(columns=misc_drop),
    on="Player",
    how="left",
    suffixes=("", "_misc"),
)

# -----------------------------
# Inspect final shape
# -----------------------------
print("Final master shape:", master.shape)
master.head()


Final master shape: (25, 60)


,Player,Nation,Pos,Age,MP,Min,Mn/MP,Min%,90s,Starts,...,2CrdY,Fls,Fld,Off,Crs,Int,TklW,PKwon,PKcon,OG
0,Rio Ferdinand,eng ENG,DF,28,35,3045,87,89.0,33.8,35,...,NaN,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Wes Brown,eng ENG,DF,27,36,3030,84,88.6,33.7,34,...,NaN,51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Patrice Evra,fr FRA,DF,26,33,2820,85,82.5,31.3,33,...,NaN,38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Nemanja Vidić,rs SRB,DF,25,32,2665,83,77.9,29.6,32,...,NaN,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Cristiano Ronaldo,pt POR,"FW,MF",22,34,2747,81,80.3,30.5,31,...,NaN,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
from pathlib import Path

PROJECT_ROOT = Path("..").resolve()
DATA_PROCESSED = PROJECT_ROOT / "data" / "processed"

DATA_PROCESSED.mkdir(parents=True, exist_ok=True)

master.to_csv(DATA_PROCESSED / "manutd_2007_08_master.csv", index=False)
master.shape, master.head()


((25, 60),
               Player   Nation    Pos  Age  MP   Min  Mn/MP  Min%   90s  \
 0      Rio Ferdinand  eng ENG     DF   28  35  3045     87  89.0  33.8   
 1          Wes Brown  eng ENG     DF   27  36  3030     84  88.6  33.7   
 2       Patrice Evra   fr FRA     DF   26  33  2820     85  82.5  31.3   
 3      Nemanja Vidić   rs SRB     DF   25  32  2665     83  77.9  29.6   
 4  Cristiano Ronaldo   pt POR  FW,MF   22  34  2747     81  80.3  30.5   
 
    Starts  ...  2CrdY  Fls  Fld  Off  Crs  Int  TklW  PKwon  PKcon  OG  
 0      35  ...    NaN   15  NaN  NaN  NaN  NaN   NaN    NaN    NaN NaN  
 1      34  ...    NaN   51  NaN  NaN  NaN  NaN   NaN    NaN    NaN NaN  
 2      33  ...    NaN   38  NaN  NaN  NaN  NaN   NaN    NaN    NaN NaN  
 3      32  ...    NaN   30  NaN  NaN  NaN  NaN   NaN    NaN    NaN NaN  
 4      31  ...    NaN   12  NaN  NaN  NaN  NaN   NaN    NaN    NaN NaN  
 
 [5 rows x 60 columns])